In [ ]:
import spacy
import pandas as pd
from nltk.corpus import stopwords
import nltk


In [ ]:
df = pd.read_excel("/content/max_days.xlsx")

# Load the English language model
nlp = spacy.load("en_core_web_sm")

#split the two columns I want to tokenize
primary_measure = df['primary_measure']

secondary_measure = df['secondary_measure']


In [ ]:

# Download NLTK stopwords (if not already downloaded)
nltk.download('stopwords')

# Load spaCy model
nlp = spacy.load("en_core_web_sm")

# Get NLTK English stopwords
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    # Process text with spaCy
    doc = nlp(text)

    # Lemmatize and remove punctuation and whitespace
    tokens = [token.lemma_.lower() for token in doc if not token.is_punct and not token.is_space]

    # Remove stopwords
    tokens = [token for token in tokens if token not in stop_words]

    return tokens


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# Apply preprocess_text function to primary_measure column
primary_measure = primary_measure = df['primary_measure'].apply(preprocess_text)

# Apply preprocess_text function to secondary_measure column
secondary_measure = df['secondary_measure'].apply(preprocess_text)


In [ ]:
# Combine preprocessed texts into a single list of strings
preprocessed_texts = [' '.join(tokens) for tokens in primary_measure]
# Initialize TF-IDF vectorizer
vectorizer = TfidfVectorizer()

# TF-IDF vectorization
X = vectorizer.fit_transform(preprocessed_texts)

# Convert sparse matrix to DataFrame (optional for visualization)
tfidf_df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())

# Display TF-IDF DataFrame
print(tfidf_df)

       03   06  06dec2019  093   10       100  1007  100x10  101  106  ...  \
0     0.0  0.0        0.0  0.0  0.0  0.000000   0.0     0.0  0.0  0.0  ...   
1     0.0  0.0        0.0  0.0  0.0  0.000000   0.0     0.0  0.0  0.0  ...   
2     0.0  0.0        0.0  0.0  0.0  0.000000   0.0     0.0  0.0  0.0  ...   
3     0.0  0.0        0.0  0.0  0.0  0.000000   0.0     0.0  0.0  0.0  ...   
4     0.0  0.0        0.0  0.0  0.0  0.000000   0.0     0.0  0.0  0.0  ...   
...   ...  ...        ...  ...  ...       ...   ...     ...  ...  ...  ...   
1629  0.0  0.0        0.0  0.0  0.0  0.000000   0.0     0.0  0.0  0.0  ...   
1630  0.0  0.0        0.0  0.0  0.0  0.324641   0.0     0.0  0.0  0.0  ...   
1631  0.0  0.0        0.0  0.0  0.0  0.000000   0.0     0.0  0.0  0.0  ...   
1632  0.0  0.0        0.0  0.0  0.0  0.000000   0.0     0.0  0.0  0.0  ...   
1633  0.0  0.0        0.0  0.0  0.0  0.000000   0.0     0.0  0.0  0.0  ...   

      york      ypn0      ypt0  ypt0n0   yr  zinc  zoladex  zon

In [ ]:
y = df['study_duration_days'].values

# Initialize imputer to replace NaNs with the mean
imputer = SimpleImputer(strategy='mean')

# Fit the imputer on your data and transform 'y' to replace NaNs
y_imputed = imputer.fit_transform(y.reshape(-1, 1)).ravel()  # Reshape for single feature


In [ ]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_imputed, test_size=0.2, random_state=42)


In [ ]:
from sklearn.linear_model import LinearRegression
#Base
# Initialize the regression model
model = LinearRegression()

# Fit the model on the training data
model.fit(X_train, y_train)

# Predict on the test data
y_pred = model.predict(X_test)

# Evaluate the model
from sklearn.metrics import mean_squared_error, r2_score

print("Mean Squared Error:", mean_squared_error(y_test, y_pred))
print("R-squared:", r2_score(y_test, y_pred))


Mean Squared Error: 1641143.5851114725
R-squared: -1.2828049421680885


In [ ]:
#idea one, just trying stuff
# Example of trying Ridge Regression with hyperparameter tuning
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV

# Initialize Ridge Regression model
ridge_model = Ridge()

# Define parameter grid for GridSearchCV
param_grid = {'alpha': [0.1, 1.0, 10.0]}  # values for regularization parameter

# Perform GridSearchCV for hyperparameter tuning
grid_search = GridSearchCV(ridge_model, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

# Best model after hyperparameter tuning
best_model = grid_search.best_estimator_

# Evaluate the best model
y_pred = best_model.predict(X_test)
print("Mean Squared Error:", mean_squared_error(y_test, y_pred))
print("R-squared:", r2_score(y_test, y_pred))


Mean Squared Error: 499498.320842505
R-squared: 0.30520568354384325


In [ ]:
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
#idea two MLE guy from AZ

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize SelectKBest with f_regression for continuous target
selector = SelectKBest(score_func=f_regression, k=100)  # select top 100 features

# Fit selector to your data and transform both training and testing data
X_train_selected = selector.fit_transform(X_train, y_train)  # Use y_train here
X_test_selected = selector.transform(X_test)  # Transform the test data as well

# Train a regression model with filtered features
model = LinearRegression()
model.fit(X_train_selected, y_train)  # Use y_train for training

# Predict on the transformed test set
y_pred = model.predict(X_test_selected)  # Use X_test_selected for prediction

# Evaluate the model
print("Mean Squared Error:", mean_squared_error(y_test, y_pred))
print("R-squared:", r2_score(y_test, y_pred))

Mean Squared Error: 602518.8530926044
R-squared: 0.16190574178442685
